To run this notebook, start Jupyter as follows from the Django project root: <br>
(https://medium.com/ayuth/how-to-use-django-in-jupyter-notebook-561ea2401852)

`python manage.py shell_plus --notebook`

<br>
NOTE:  You will need to changed the kernel from menu: `Kernel` > `Change kernel` > `Django Shell-Plus`

In [1]:
import os
import logging
import pandas as pd
import geopandas as gpd
import datetime

# for census data used in Bubble maps
from census import Census

# import variables in the local .env file
from dotenv import load_dotenv
load_dotenv(verbose=False)

True

In [2]:
# avoid chained assignment warnings
pd.options.mode.chained_assignment = None

In [3]:
!pwd

/Users/markmcdonald/Desktop/marks-covid-tracker/tracker/notebooks


In [4]:
# GLOBAL VARIABLES
FILE_PATH = os.path.join('..', 'covid_tracker', 'COVID-19', 'csse_covid_19_data', 'csse_covid_19_time_series')

territories = ['American Samoa', 'Guam', 'Northern Mariana Islands', 'Mariana Islands',
               'Puerto Rico', 'Virgin Islands', 'Diamond Princess', 'Grand Princess']

political_affiliations = {'American Samoa': 'na',
                          'Guam': 'na',
                          'Northern Mariana Islands': 'na',
                          'Puerto Rico': 'na',
                          'Virgin Islands': 'na',
                          'Alabama': 'red',
                          'Alaska': 'red',
                          'Arizona': 'red',
                          'Arkansas': 'red',
                          'California': 'blue',
                          'Colorado': 'blue',
                          'Connecticut': 'blue',
                          'Delaware': 'blue',
                          'District of Columbia': 'blue',
                          'Florida': 'red',
                          'Georgia': 'red',
                          'Hawaii': 'blue',
                          'Idaho': 'red',
                          'Illinois': 'blue',
                          'Indiana': 'red',
                          'Iowa': 'red',
                          'Kansas': 'blue',
                          'Kentucky': 'blue',
                          'Louisiana': 'blue',
                          'Maine': 'blue',
                          'Maryland': 'red',
                          'Massachusetts': 'red',
                          'Michigan': 'blue',
                          'Minnesota': 'blue',
                          'Mississippi': 'red',
                          'Missouri': 'red',
                          'Montana': 'blue',
                          'Nebraska': 'red',
                          'Nevada': 'blue',
                          'New Hampshire': 'red',
                          'New Jersey': 'blue',
                          'New Mexico': 'blue',
                          'New York': 'blue',
                          'North Carolina': 'blue',
                          'North Dakota': 'red',
                          'Ohio': 'red',
                          'Oklahoma': 'red',
                          'Oregon': 'blue',
                          'Pennsylvania': 'blue',
                          'Rhode Island': 'blue',
                          'South Carolina': 'red',
                          'South Dakota': 'red',
                          'Tennessee': 'red',
                          'Texas': 'red',
                          'Utah': 'red',
                          'Vermont': 'red',
                          'Virginia': 'blue',
                          'Washington': 'blue',
                          'West Virginia': 'red',
                          'Wisconsin': 'blue',
                          'Wyoming': 'red',
                          'Diamond Princess': 'na',
                          'Grand Princess': 'na'}

In [5]:
try:
    from django.http import JsonResponse

except Exception as e:
    print("NOT LOADED:  start notebook with:\n")
    print("\tpython manage.py shell_plus --notebook")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [6]:
!pwd

/Users/markmcdonald/Desktop/marks-covid-tracker/tracker/notebooks


In [7]:
import git
import json
from django.http import JsonResponse

def refresh_git(request):
    """
    Triggers a pull of the Git repository that hold the data used in plots.

    :param request: The HTML request which is provided by Django when the route is called.  No values of the request are used in this function.

    :return: A Json formatted text response that includes the git activity of the request to pull data.
    """
    try:
        g = git.cmd.Git('../covid_tracker/COVID-19')
        rv = g.pull()
    except Exception as e:
        rv = "Git sync in process..."

    print(rv)
#     return JsonResponse(json.dumps(rv), safe=False)

In [8]:
refresh_git(None)

Updating 4c6b397c9..bceb5d0f8
Fast-forward
 csse_covid_19_data/README.md                       |    6 +-
 .../csse_covid_19_daily_reports/04-17-2021.csv     | 3984 ++++++++++++
 .../csse_covid_19_daily_reports_us/04-17-2021.csv  |   59 +
 .../time_series_covid19_confirmed_US.csv           | 6686 ++++++++++----------
 .../time_series_covid19_confirmed_global.csv       |  550 +-
 .../time_series_covid19_deaths_US.csv              | 6686 ++++++++++----------
 .../time_series_covid19_deaths_global.csv          |  550 +-
 .../time_series_covid19_recovered_global.csv       |  520 +-
 8 files changed, 11544 insertions(+), 7497 deletions(-)
 create mode 100644 csse_covid_19_data/csse_covid_19_daily_reports/04-17-2021.csv
 create mode 100644 csse_covid_19_data/csse_covid_19_daily_reports_us/04-17-2021.csv


In [9]:
# GET CSV DATA, CLEAN DATA, AND PLACE INTO DICTIONARY FOR EASY ACCESS
def get_dataframe(dataset, file_path: str = FILE_PATH) -> dict:
    """
    Support function that will retrieve a dictionary which includes a dataset in a dataframe format for simplified downstream sorting and filtering.

    :param dataset: The string name of the dataset for which data is retrieved ('recovered_global', 'deaths_global', 'deaths_US', 'confirmed_global', 'confirmed_US')
    :param file_path: (optional) The root filepath where the applicable data is stored.

    :return: Returns a dictionary with the dataframe along with applicable parameters useful in handling the dataset. The dictionary container the following key-values {'df': <dataframe>, 'attr_cols': <list of names of attribute columns>, 'date_cols_text': <list of date columns in text format>, 'date_cols_dates': <list of date columns in datetime format>}
    """
    file_names = {
        'recovered_global': 'time_series_covid19_recovered_global.csv',
        'deaths_global': 'time_series_covid19_deaths_global.csv',
        'deaths_US': 'time_series_covid19_deaths_US.csv',
        'confirmed_global': 'time_series_covid19_confirmed_global.csv',
        'confirmed_US': 'time_series_covid19_confirmed_US.csv'
    }

    file_name = file_names.get(dataset)

    def get_pol_aff(s):
        return political_affiliations.get(s.Province_State, 'na')

    def make_county_state(s):
        return f"{s.County}, {s.Province_State}"

    f = os.path.join(file_path, file_name)
    if not os.path.isfile(f):
        logging.error(f"{f} is not a file.")

    df = pd.DataFrame()
    if os.path.isfile(f):
        df = pd.read_csv(f)
        if "Province_State" in df.columns:
            # add political affiliation
            df['political_affiliation'] = df.apply(get_pol_aff, axis=1)

            if "Admin2" in df.columns:
                df = df.rename(columns={'Admin2': 'County'})
                df['County_State'] = df.apply(make_county_state, axis=1)

            # format FIPS
            df['FIPS'] = df['UID'].astype(str).apply(lambda x: x[3:])

            # add population by county
            c = Census(os.getenv("CENSUS_API_KEY"), year=2010)
            county_population = pd.DataFrame(
                c.sf1.state_county(['NAME', 'P001001'], state_fips=Census.ALL, county_fips=Census.ALL))
            county_population.rename(
                columns={'P001001': 'population', 'state': 'state_fips', 'county': 'county_fips'}, inplace=True)
            county_population.population = county_population.population.astype(int)
            county_population['FIPS'] = county_population.state_fips + county_population.county_fips
            df = df.merge(county_population[['FIPS', 'population']], on=['FIPS'])

    else:
        if not os.path.isdir(file_path):
            logging.error(f"NOT A PATH: '{file_path}''")
        else:
            logging.error(f"NOT A FILE: '{file_name}''")

    # delete jan and feb
    cols_to_remove = []
    for c in df.columns:
        spl = c.split('/')
        if len(spl) == 3 and spl[2] == '20':
            if spl[0] == '1' or spl[0] == '2':
                cols_to_remove.append(c)
    df.drop(columns=cols_to_remove, inplace=True)

    # remove US territories
    if "Province_State" in df.columns:
        df = df[~df.Province_State.isin(territories)]

    # convert to GeoPandas if there are lat and lon coordinates
    if "Lat" in df.columns and "Long_" in df.columns:
        df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Long_, df.Lat))

    attr_cols, date_cols_text, date_cols_dates = get_column_groups(df)

    return {'df': df, 'attr_cols': attr_cols, 'date_cols_text': date_cols_text, 'date_cols_dates': date_cols_dates}

def get_column_groups(df):
    """
    Returns a tuple of 3 lists that include the columns of a dataframe split into 3 groups. 'Attribute Columns' are non-date columns in a dataframe,  'Date Columns Text' are date columns in text format and 'Date Columns Date' are date columns in datetime format.

    :param df:  The dataframe that will be evaluated and for which the column groups are returned.

    :return: A tuple of 3 lists: the attribute columns (all columns that are not dates), date columns in text format and the date columns is datetime format.
    """
    date_cols_dates = pd.to_datetime(df.columns, errors='coerce')
    date_cols_tf = [not pd.isnull(c) for c in date_cols_dates]
    date_cols_dates = date_cols_dates[date_cols_tf]
    date_cols_text = df.columns[date_cols_tf]
    attr_cols = df.columns[[not c for c in date_cols_tf]]

    return attr_cols, date_cols_text, date_cols_dates


# SUPPORT FUNCTIONS
# group by state
def get_df_by_state(_df) -> pd.DataFrame:
    """
    Support function that will return a dataframe grouped by 'Province State' where values are summed.

    :param _df:  The dataframe that will be grouped.

    :return: The grouped dataframe
    """
    return _df.groupby(by='Province_State').sum().reset_index()


# group by county in a state
def get_df_by_counties(_df, state) -> pd.DataFrame:
    """
    A support function that will return the entries in a provided dataframe that match the provided state.  The state value provided is not case sensitive.

    :param _df:  The initial dataframe that will be filtered.
    :param state: The state string name that will be used to filter the dataframe.

    :return: A dataframe that includes the entries of the provided dataframe that only include the provided state.
    """
    return _df[_df.Province_State.str.lower() == state.lower()]


def get_rankings(_df, top_n=None) -> list:
    """
    Support function that will return a list of the top n listings in a dataframe based on the most recent date.  Results are listing in descending order.  If no top_n value is provided, all items in the dataframe are sorted.

    :param _df:  The dataframe that will be filtered and sorted.
    :param top_n: (optional) An integer value that will restrict the returned list to the top n number of listings.

    :return: A list of top_n indexes from a dataframe
    """
    attr_cols, date_cols_text, date_cols_dates = get_column_groups(_df)
    a_cols = [c for c in _df.columns if c in attr_cols]

    rv_df = _df[date_cols_text].rank(ascending=False)
    rv_df = pd.concat([_df.loc[rv_df.index][a_cols], rv_df], axis=1)

    if top_n is not None:
        return rv_df.nsmallest(top_n, date_cols_text[-1]).sort_values(by=date_cols_text[-1], ascending=True)

    return rv_df.sort_values(by=date_cols_text[-1])


# get data per day
def get_by_day(_df):
    """
    Return a dataframe that contains cumulative values in date columns in a daily value format.

    :param _df:  A dataframe where the values in each date column are cumulative values.

    :return: The supplied dataframe with daily data instead of cumulative data.
    """
    _, date_cols_text, _ = get_column_groups(_df)
    daily = _df[date_cols_text] - _df[date_cols_text].shift(axis=1)
    attr_cols = set(_df.columns) - set(date_cols_text)
    daily = pd.concat([_df[attr_cols], daily], axis=1)

    return daily


def get_daily_growth_rate(_df):
    """
    Return a dataframe that includes values that represent the change rate vs the prior day for all values in date columns.

    :param _df:  A dataframe of cumulative values in date columns

    :return: A dataframe with the same columns and rows as the original dataframe, except the values are daily change rates rather than cumulative absolute values.
    """
    _, date_cols_text, _ = get_column_groups(_df)
    _df = get_by_day(_df)
    daily_rate = _df[date_cols_text] / _df[date_cols_text].shift(axis=1)
    attr_cols = set(_df.columns) - set(date_cols_text)
    daily_rate = pd.concat([_df[attr_cols], daily_rate], axis=1)
    daily_rate = daily_rate.fillna(1)
    return daily_rate




In [33]:
from bokeh.embed import json_item
from bokeh.plotting import figure
from bokeh.models import FactorRange, ColumnDataSource, HoverTool, NumeralTickFormatter
from bokeh.models.callbacks import CustomJS

from django.http import JsonResponse

# from .helpers import *


def _get_plot_data(data_type, frequency, state, county):
    # set dataframes
    if data_type == 'infections':
        df_dict = get_dataframe('confirmed_US')
    else:
        df_dict = get_dataframe('deaths_US')

    # _, date_cols_text, date_cols_dates = get_column_groups(df)

    df = df_dict['df']
    date_cols_text = df_dict['date_cols_text']
    date_cols_dates = df_dict['date_cols_dates']

    if frequency == 'daily':
        all_data = get_by_day(df)
    else:
        all_data = df.copy()

    if state == 'United States':
        plot_data = all_data.sum()[date_cols_text].values
    else:
        if county == 'All':
            plot_data = all_data[all_data.Province_State == state].sum()[date_cols_text].values
        else:
            plot_data = all_data[(all_data.Province_State == state) & (all_data.County == county)].sum()[
                date_cols_text].values

    # setup x axis groupings
    factors = [(str(c.year), c.month_name(), str(c.day)) for c in date_cols_dates]

    return plot_data, factors


def plot_state_chart(request, state="United States", county='All', frequency='daily', data_type='infections', rolling_window=14):
    """
    Plots the values for a selected state, county or the entire United States.

    :param request:  The HTML request that should include values for 'state', 'county', 'frequency', 'data_type' and 'rolling_window'.  See the parameter descriptions below for contraints and defaults for each parameter.
    :param state: (optional) Default 'Massachusetts'.  The name of the state to plot.
    :param county: (optional) Default 'All'.  The county to plot or 'All' counties of a state of value is 'All'.
    :param frequency: (optional) Default 'daily'.  Whether to plot daily or cumulative values.
    :param data_type: (optional) Default 'infections'.  Plot 'infections' data or 'deaths' data.
    :param rolling_window: (optional) Default 14.  The number of days to use when drawing the daily average line in the plot.

    :return: A Bokeh JSON formatted plot that can be handled by JavaScript for HTML presentation.
    """
    if request is not None:
        state = request.GET.get('state', 'United States')
        county = request.GET.get('county', 'All')
        frequency = request.GET.get('frequency', 'daily')
        data_type = request.GET.get('data_type', 'infections').lower()
        rolling_window = int(request.GET.get('rolling_window', 14))

    state = ' '.join([word.capitalize() for word in state.split(' ')])
    county = county.capitalize()

    plot_data, factors = _get_plot_data(data_type, frequency, state, county)

    # # setup x axis groupings
    # factors = [(c.month_name(), str(c.day)) for c in DATE_COLS_DATES]

    # setup Hover tool
    hover = HoverTool()
    hover.tooltips = [
        ("Date", "@date"),
        (data_type.capitalize(), "@val{0,0}"),
        (f"{rolling_window}-day Avg", "@rolling_avg{0,0.0}")
    ]

    # setup figure
    p = figure(x_range=FactorRange(*factors), sizing_mode='stretch_both',  # plot_height=500, plot_width=900,
               y_axis_type='linear', y_axis_label=data_type, output_backend="webgl",
               toolbar_location=None, tools=[hover],
               title=f"{state} New {data_type.capitalize()}{' by Day' if frequency == 'daily' else ''}")
    p.title.text_font_size = '12pt'
    p.yaxis.formatter = NumeralTickFormatter(format="0,000")

    source = ColumnDataSource(
        data=dict(date=factors, val=plot_data, rolling_avg=pd.Series(plot_data).rolling(rolling_window).mean().values))

    b = p.vbar(x='date', top='val', source=source, color='red', width=.5)

    if frequency == 'daily':
        l = p.line(x='date', y='rolling_avg', source=source, color='black', width=3, legend_label=f"{rolling_window}-Day Rolling Average")
        p.legend.location = 'top_left'

    p.xaxis.major_label_orientation = 1
    p.xaxis.group_text_font_size = "10pt"  # months size
    p.xaxis.major_label_text_font_size = "6pt"  # date size
    p.yaxis.major_label_orientation = 1
    p.xgrid.grid_line_color = None

    callback = CustomJS(args=dict(source=source), code="""

        // JavaScript code goes here

        var updated_data; 

        // the model that triggered the callback is cb_obj:
        fetch(cb_obj.value)
        .then( response = return response.json() )
        .then( x => udpated_data = x ) 

        // models passed as args are automagically available
        source.data = updated_data;

        """)
    
    return p
#     return JsonResponse(json_item(p))

In [34]:
from bokeh.plotting import show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [35]:
_p = plot_state_chart(None)

In [36]:
show(_p)

In [25]:
plot_data, factors = _get_plot_data('infections', "daily", "United States", "All")

In [28]:
len(plot_data)

413

In [32]:
factors

[('March', '1', '2020'),
 ('March', '2', '2020'),
 ('March', '3', '2020'),
 ('March', '4', '2020'),
 ('March', '5', '2020'),
 ('March', '6', '2020'),
 ('March', '7', '2020'),
 ('March', '8', '2020'),
 ('March', '9', '2020'),
 ('March', '10', '2020'),
 ('March', '11', '2020'),
 ('March', '12', '2020'),
 ('March', '13', '2020'),
 ('March', '14', '2020'),
 ('March', '15', '2020'),
 ('March', '16', '2020'),
 ('March', '17', '2020'),
 ('March', '18', '2020'),
 ('March', '19', '2020'),
 ('March', '20', '2020'),
 ('March', '21', '2020'),
 ('March', '22', '2020'),
 ('March', '23', '2020'),
 ('March', '24', '2020'),
 ('March', '25', '2020'),
 ('March', '26', '2020'),
 ('March', '27', '2020'),
 ('March', '28', '2020'),
 ('March', '29', '2020'),
 ('March', '30', '2020'),
 ('March', '31', '2020'),
 ('April', '1', '2020'),
 ('April', '2', '2020'),
 ('April', '3', '2020'),
 ('April', '4', '2020'),
 ('April', '5', '2020'),
 ('April', '6', '2020'),
 ('April', '7', '2020'),
 ('April', '8', '2020'),
 ('